[**Open In Colab**](https://colab.research.google.com/github/HassanAlgoz/agentic-ai-systems/blob/main/Lessons/L01/02_subagents.ipynb)

# Build a personal assistant with subagents

## Overview

The **supervisor pattern** is a [multi-agent](https://docs.langchain.com/oss/python/langchain/multi-agent) architecture where a central supervisor agent coordinates specialized worker agents. This approach excels when tasks require different types of expertise. Rather than building one agent that manages tool selection across domains, you create focused specialists coordinated by a supervisor who understands the overall workflow.

In this tutorial, you'll build a personal assistant system that demonstrates these benefits through a realistic workflow. The system will coordinate two specialists with fundamentally different responsibilities:

* A **calendar agent** that handles scheduling, availability checking, and event management.
* An **email agent** that manages communication, drafts messages, and sends notifications.

### Why use a supervisor?

Multi-agent architectures allow you to partition [tools](https://docs.langchain.com/oss/python/langchain/tools) across workers, each with their own individual prompts or instructions. Consider an agent with direct access to all calendar and email APIs: it must choose from many similar tools, understand exact formats for each API, and handle multiple domains simultaneously. If performance degrades, it may be helpful to separate related tools and associated prompts into logical groups (in part to manage iterative improvements).


### Understanding the architecture

![Architecture](./assets/subagents_arch_detailed.png)

Your system has three layers. The bottom layer contains rigid API tools that require exact formats. The middle layer contains sub-agents that accept natural language, translate it to structured API calls, and return natural language confirmations. The top layer contains the supervisor that routes to high-level capabilities and synthesizes results.

This separation of concerns provides several benefits: each layer has a focused responsibility, you can add new domains without affecting existing ones, and you can test and iterate on each layer independently.

### Components

We will need to select a chat model from LangChain's suite of integrations:


#### Select a chat model

👉 Read the [OpenAI chat model integration docs](https://docs.langchain.com/oss/python/integrations/chat/openai/)

```shell
!pip install "langchain[openai]"
```

In [ ]:
import os
from google.colab import userdata

# We use OpenRouter for the agent — add OPENROUTER_API_KEY to Colab Secrets (key icon in left sidebar)
# Get your key at https://openrouter.ai/keys
os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

# https://openrouter.ai/nvidia/nemotron-3-nano-30b-a3b:free
model_nemotron3_nano = ChatOpenAI(
    model="nvidia/nemotron-3-nano-30b-a3b:free",
    temperature=0,
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)

model = model_nemotron3_nano

## 1. Define tools

- Start by defining the tools that require structured inputs.
- In real applications, these would call actual APIs (Google Calendar, SendGrid, etc.).
- For this tutorial, you'll use stubs to demonstrate the pattern.

In [6]:
from langchain.tools import tool

In [7]:
@tool
def create_calendar_event(
    title: str,
    start_time: str,       # ISO format: "2024-01-15T14:00:00"
    end_time: str,         # ISO format: "2024-01-15T15:00:00"
    attendees: list[str],  # email addresses
    location: str = ""
) -> str:
    """Create a calendar event. Requires exact ISO datetime format."""
    # Stub: In practice, this would call Google Calendar API, Outlook API, etc.
    return f"Event created: {title} from {start_time} to {end_time} with {len(attendees)} attendees"

In [8]:
@tool
def send_email(
    to: list[str],  # email addresses
    subject: str,
    body: str,
    cc: list[str] = []
) -> str:
    """Send an email via email API. Requires properly formatted addresses."""
    # Stub: In practice, this would call SendGrid, Gmail API, etc.
    return f"Email sent to {', '.join(to)} - Subject: {subject}"


In [9]:
@tool
def get_available_time_slots(
    attendees: list[str],
    date: str,  # ISO format: "2024-01-15"
    duration_minutes: int
) -> list[str]:
    """Check calendar availability for given attendees on a specific date."""
    # Stub: In practice, this would query calendar APIs
    return ["09:00", "14:00", "16:00"]

## 2. Create specialized sub-agents

Next, we'll create specialized sub-agents that handle each domain.


### Create a calendar agent

- The calendar agent understands natural language scheduling requests and translates them into precise API calls.
- It handles date parsing, availability checking, and event creation.

In [ ]:

from langchain.agents import create_agent

CALENDAR_AGENT_PROMPT = (
    "You are a calendar scheduling assistant. "
    "Parse natural language scheduling requests (e.g., 'next Tuesday at 2pm') "
    "into proper ISO datetime formats. "
    "Use get_available_time_slots to check availability when needed. "
    "Use create_calendar_event to schedule events. "
    "Always confirm what was scheduled in your final response."
)

calendar_agent = create_agent(
    model,
    tools=[
        create_calendar_event,
        get_available_time_slots
    ],
    system_prompt=CALENDAR_AGENT_PROMPT,
)

Test the calendar agent to see how it handles natural language scheduling:

In [ ]:
from langchain.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

In [ ]:
query = "Schedule a team meeting next Tuesday at 2pm for 1 hour"
input_ = {"messages": [{"role": "user", "content": query}]}

for step in calendar_agent.stream(input_):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_VPij7Wlqm5MTct1VYogczdmM)
 Call ID: call_VPij7Wlqm5MTct1VYogczdmM
  Args:
    title: Team meeting
    start_time: 2026-02-24T14:00:00
    end_time: 2026-02-24T15:00:00
    attendees: []
    location:
================================= Tool Message =================================
Name: create_calendar_event

Event created: Team meeting from 2026-02-24T14:00:00 to 2026-02-24T15:00:00 with 0 attendees
================================== Ai Message ==================================

All set. I’ve scheduled the Team meeting for Tuesday, February 24, 2026, from 2:00 PM to 3:00 PM (local time). Attendees: none. Location: none.

Would you like me to add attendees and/or a location (or conferencing details) and send invites?


The agent parses "next Tuesday at 2pm" into ISO format ("2024-01-16T14:00:00"), calculates the end time, calls `create_calendar_event`, and returns a natural language confirmation.

### Create an email agent

- The email agent handles message composition and sending.
- It focuses on extracting recipient information, crafting appropriate subject lines and body text, and managing email communication.

In [12]:
EMAIL_AGENT_PROMPT = (
    "You are an email assistant. "
    "Compose professional emails based on natural language requests. "
    "Extract recipient information and craft appropriate subject lines and body text. "
    "Use send_email to send the message. "
    "Always confirm what was sent in your final response."
)

email_agent = create_agent(
    model,
    tools=[send_email],
    system_prompt=EMAIL_AGENT_PROMPT,
)

Test the email agent with a natural language request:

In [13]:
query = "Send the design team a reminder about reviewing the new mockups"
input_ = {"messages": [{"role": "user", "content": query}]}

for step in email_agent.stream(input_):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================

Here’s a ready-to-send draft. I’ve assumed a Design Team distribution list. If you want a different recipient, let me know.

Recipients: design-team@yourcompany.com
Subject: Reminder: Please review the new mockups
Body:
Hi Design Team,

This is a friendly reminder to review the new mockups for the [Project/Feature]. Your feedback is important to ensure we stay aligned with requirements and timelines.

Please share your comments by [Due Date] or let me know if you need more time. You can access the mockups here: [Link to mockups].

If you’ve already provided feedback, thank you and please disregard this message.

Best regards,
[Your Name]
[Your Title]
[Department]

Would you like me to send this now as-is, or would you like me to replace the placeholders with the actual project name, due date, link, and your signature? If you confirm, I’ll send it.


- The agent infers the recipient from the informal request, crafts a professional subject line and body, calls `send_email`, and returns a confirmation.
- Each sub-agent has a narrow focus with domain-specific tools and prompts, allowing it to excel at its specific task.

## 3. Wrap sub-agents as tools

- Now wrap each sub-agent as a tool that the supervisor can invoke.
- This is the key architectural step that creates the layered system.
- The supervisor will see high-level tools like `"schedule_event"`, not low-level tools like `"create_calendar_event"`.

In [14]:
@tool
def schedule_event(request: str) -> str:
    """Schedule calendar events using natural language.

    Use this when the user wants to create, modify, or check calendar appointments.
    Handles date/time parsing, availability checking, and event creation.

    Input: Natural language scheduling request (e.g., 'meeting with design team
    next Tuesday at 2pm')
    """
    result = calendar_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text

In [15]:
@tool
def manage_email(request: str) -> str:
    """Send emails using natural language.

    Use this when the user wants to send notifications, reminders, or any email
    communication. Handles recipient extraction, subject generation, and email
    composition.

    Input: Natural language email request (e.g., 'send them a reminder about
    the meeting')
    """
    result = email_agent.invoke({
        "messages": [{"role": "user", "content": request}]
    })
    return result["messages"][-1].text


- The **tool descriptions (docstring)** help the supervisor decide when to use each tool, so make them clear and specific.
- We return only the sub-agent's final response, as the supervisor doesn't need to see intermediate reasoning or tool calls.

## 4. Create the supervisor agent

- Now create the supervisor that orchestrates the sub-agents.
- The supervisor only sees high-level tools and makes routing decisions at the domain level, not the individual API level.

In [17]:
SUPERVISOR_PROMPT = (
    "You are a helpful personal assistant. "
    "You can schedule calendar events and send emails. "
    "Break down user requests into appropriate tool calls and coordinate the results. "
    "When a request involves multiple actions, use multiple tools in sequence."
)

supervisor_agent = create_agent(
    model,
    tools=[
        schedule_event,
        manage_email
    ],
    system_prompt=SUPERVISOR_PROMPT,
)

## 5. Use the supervisor

Now test your complete system with complex requests that require coordination across multiple domains:


### Example 1: Simple single-domain request

In [18]:
query = "Schedule a team standup for tomorrow at 9am"
input_ = {"messages": [{"role": "user", "content": query}]}

for step in supervisor_agent.stream(input_):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  schedule_event (call_gtoo8MGcOP1cOyGS8hm1rkRM)
 Call ID: call_gtoo8MGcOP1cOyGS8hm1rkRM
  Args:
    request: Schedule a team standup tomorrow at 9am
================================= Tool Message =================================
Name: schedule_event

I can schedule that. A couple of details to confirm:

- Date: Tomorrow is 2026-02-19. Is that correct?
- Duration: What duration should I use? (default 15 minutes)
- Attendees: Who should attend the standup? (e.g., list of names, or simply "Team")
- Location/ conferencing: Should this be virtual (and which platform) or in-person? If virtual, do you want me to create a meeting link?
- Timezone: Should I use your calendar’s default timezone, or specify a different one?

If you’d like, I can proceed with defaults (9:00–9:15 local time, attendees: Team, virtual meeting link to be determined) once you confirm.
================================== Ai Mes

The supervisor identifies this as a calendar task, calls `schedule_event`, and the calendar agent handles date parsing and event creation.

::: {.callout-tip}
For full transparency into the information flow, including prompts and responses for each chat model call, check out the [LangSmith trace](https://smith.langchain.com/public/91a9a95f-fba9-4e84-aff0-371861ad2f4a/r) for the above run.
:::

### Example 2: Complex multi-domain request

In [ ]:
query = (
    "Schedule a meeting with the design team next Tuesday at 2pm for 1 hour, "
    "and send them an email reminder about reviewing the new mockups."
)
input_ = {"messages": [{"role": "user", "content": query}]}

for step in supervisor_agent.stream(input_):
    for update in step.values():
        for message in update.get("messages", []):
            message.pretty_print()

- The supervisor recognizes this requires both calendar and email actions, calls `schedule_event` for the meeting, then calls `manage_email` for the reminder.
- Each sub-agent completes its task, and the supervisor synthesizes both results into a coherent response.

::: {.callout-tip}
Refer to the [LangSmith trace](https://smith.langchain.com/public/95cd00a3-d1f9-4dba-9731-7bf733fb6a3c/r) to see the detailed information flow for the above run, including individual chat model prompts and responses.
:::

## Key takeaways

- The supervisor pattern creates layers of abstraction where each layer has a clear responsibility.
- When designing a supervisor system, start with clear domain boundaries and give each sub-agent focused tools and prompts.
- Write clear tool descriptions for the supervisor, test each layer independently before integration, and control information flow based on your specific needs.

Use the supervisor pattern when you have multiple distinct domains (calendar, email, CRM, database), each domain has multiple tools or complex logic, you want centralized workflow control, and sub-agents don't need to converse directly with users.

For simpler cases with just a few tools, use a single agent. When agents need to have conversations with users, use [handoffs](https://docs.langchain.com/oss/python/langchain/multi-agent/handoffs) instead. For peer-to-peer collaboration between agents, consider other multi-agent patterns.

## Activity

**Over to you:** recreate the supervisor pattern on a different problem domain (other than calendar & emails). You may stub API calls or, better yet, use actual ones!